In [7]:
""" 
    Starter code for exploring the Enron dataset (emails + finances);
    loads up the dataset (pickled dict of dicts).

    The dataset has the form:
    enron_data["LASTNAME FIRSTNAME MIDDLEINITIAL"] = { features_dict }

    {features_dict} is a dictionary of features associated with that person.
    You should explore features_dict as part of the mini-project,
    but here's an example to get you started:

    enron_data["SKILLING JEFFREY K"]["bonus"] = 5600000
    
"""

import pickle
import pprint as pp
from poi_email_addresses import poiEmails
import operator


def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False


poi_emails = poiEmails()

with open("../final_project/final_project_dataset.pkl", "r") as f:
    enron_data = pickle.load(f)
    
    poi = set()
    for empl in enron_data:
        if enron_data[empl]['poi'] == True:poi.add(empl)
    
    
    top_of_asshole = dict()
    hided_bitches = dict()
    hided_bitches_and_POI = dict()
    for asshole in enron_data:
        if not isnan(enron_data[asshole]['total_payments']):
            top_of_asshole[asshole] = (enron_data[asshole]['total_payments'],\
                                       enron_data[asshole]['salary'], enron_data[asshole]['bonus'])
        elif enron_data[asshole]['poi'] == True:
            hided_bitches[asshole] = enron_data[asshole]['total_payments']
            hided_bitches_and_POI[asshole] = enron_data[asshole]['poi']
        else: hided_bitches[asshole] = enron_data[asshole]['total_payments']
    top_of_asshole = sorted(top_of_asshole.items(), key=operator.itemgetter(1), reverse=True)
    
    
    max_value = dict()
    key_mv = 'salary'
    for asshole in enron_data:
        if not isnan(enron_data[asshole][key_mv]):
            max_value[asshole] = enron_data[asshole][key_mv]
    max_value.pop('TOTAL')
    print max(max_value.values())
    print min(max_value.values())
    
    
    salary = dict()
    mail = dict()
    for empl in enron_data:
        if not isnan(enron_data[empl]['salary']):
            salary[empl] = enron_data[empl]['salary']
        if not isnan(enron_data[empl]['email_address']):
            mail[empl] = enron_data[empl]['email_address']
    salary = sorted(salary.items(), key=operator.itemgetter(1), reverse=True)
    
    
    print "nb of empl. =", len(enron_data)
    print "nb of field/empl. =", len(enron_data['LAY KENNETH L'])
    print "nb of poi =", len(poi)
    print "James Prentice / total stock value = ",\
        round(enron_data['PRENTICE JAMES']['total_stock_value'] / 10**6., 2), "M$"
    print "Wesley Colwell / from this person to poi = ",\
        enron_data['COLWELL WESLEY']['from_this_person_to_poi'], "mails"
    print "Jeffrey K Skilling / exercised stock options = ",\
        round(enron_data['SKILLING JEFFREY K']['exercised_stock_options'] / 10**6., 2), "M$"
    print "nb of empl. with salary = {}/{}, nb of empl. with email adress = {}/{}"\
        .format(len(salary), len(enron_data), len(mail), len(enron_data))
    print "nb of empl. without To. pay. data = {}, %age = {}%"\
        .format(len(hided_bitches), round((float(len(hided_bitches))/len(enron_data))*100, 2))
    print "nb of POI without To. pay. data = {}, %age = {}%"\
        .format(len(hided_bitches_and_POI), round((float(len(hided_bitches_and_POI))/len(enron_data))*100, 2))
    
    
    #pp.pprint(max_value)
    #pp.pprint(hided_bitches_and_POI)
    #pp.pprint(enron_data)

1111258
477
nb of empl. = 146
nb of field/empl. = 21
nb of poi = 18
James Prentice / total stock value =  1.1 M$
Wesley Colwell / from this person to poi =  11 mails
Jeffrey K Skilling / exercised stock options =  19.25 M$
nb of empl. with salary = 95/146, nb of empl. with email adress = 111/146
nb of empl. without To. pay. data = 21, %age = 14.38%
nb of POI without To. pay. data = 0, %age = 0.0%


In [31]:
#PAS ENCORE UTILISE A VOIR EN LECON 7

""" 
    A general tool for converting data from the
    dictionary format to an (n x k) python list that's 
    ready for training an sklearn algorithm

    n--no. of key-value pairs in dictonary
    k--no. of features being extracted

    dictionary keys are names of persons in dataset
    dictionary values are dictionaries, where each
        key-value pair in the dict is the name
        of a feature, and its value for that person

    In addition to converting a dictionary to a numpy 
    array, you may want to separate the labels from the
    features--this is what targetFeatureSplit is for

    so, if you want to have the poi label as the target,
    and the features you want to use are the person's
    salary and bonus, here's what you would do:

    feature_list = ["poi", "salary", "bonus"] 
    data_array = featureFormat( data_dictionary, feature_list )
    label, features = targetFeatureSplit(data_array)

    the line above (targetFeatureSplit) assumes that the
    label is the _first_ item in feature_list--very important
    that poi is listed first!
"""


import numpy as np

def featureFormat( dictionary, features, remove_NaN=True, remove_all_zeroes=True,\
                    remove_any_zeroes=False, sort_keys = False):
    """ convert dictionary to numpy array of features
        remove_NaN = True will convert "NaN" string to 0.0
        remove_all_zeroes = True will omit any data points for which
            all the features you seek are 0.0
        remove_any_zeroes = True will omit any data points for which
            any of the features you seek are 0.0
        sort_keys = True sorts keys by alphabetical order. Setting the value as
            a string opens the corresponding pickle file with a preset key
            order (this is used for Python 3 compatibility, and sort_keys
            should be left as False for the course mini-projects).
        NOTE: first feature is assumed to be 'poi' and is not checked for
            removal for zero or missing values.
    """


    return_list = []

    # Key order - first branch is for Python 3 compatibility on mini-projects,
    # second branch is for compatibility on final project.
    if isinstance(sort_keys, str):
        import pickle
        keys = pickle.load(open(sort_keys, "rb"))
    elif sort_keys:
        keys = sorted(dictionary.keys())
    else:
        keys = dictionary.keys()

    for key in keys:
        tmp_list = []
        for feature in features:
            try:
                dictionary[key][feature]
            except KeyError:
                print "error: key ", feature, " not present"
                return
            value = dictionary[key][feature]
            if value=="NaN" and remove_NaN:
                value = 0
            tmp_list.append( float(value) )

        # Logic for deciding whether or not to add the data point.
        append = True
        # exclude 'poi' class as criteria.
        if features[0] == 'poi':
            test_list = tmp_list[1:]
        else:
            test_list = tmp_list
        ### if all features are zero and you want to remove
        ### data points that are all zero, do that here
        if remove_all_zeroes:
            append = False
            for item in test_list:
                if item != 0 and item != "NaN":
                    append = True
                    break
        ### if any features for a given data point are zero
        ### and you want to remove data points with any zeroes,
        ### handle that here
        if remove_any_zeroes:
            if 0 in test_list or "NaN" in test_list:
                append = False
        ### Append the data point if flagged for addition.
        if append:
            return_list.append( np.array(tmp_list) )

    return np.array(return_list)


def targetFeatureSplit( data ):
    """ 
        given a numpy array like the one returned from
        featureFormat, separate out the first feature
        and put it into its own list (this should be the 
        quantity you want to predict)

        return targets and features as separate lists

        (sklearn can generally handle both lists and numpy arrays as 
        input formats when training/predicting)
    """

    target = []
    features = []
    for item in data:
        target.append( item[0] )
        features.append( item[1:] )

    return target, features